In [ ]:
import os
import json 
import pandas as pd
from tqdm import tqdm


# Set the directory
directory = "2017_j"
target_subreddits = ["wallstreetbets"]

# Create an empty dataframe
df = pd.DataFrame(columns=["subreddit", "author", "created_utc", "parent_id", "id", "body"])


# Iterate over the files in ascending order by filename, set number of files to process
files = sorted([f for f in os.listdir(directory)])
files = files[:25]
print(files)

for file in tqdm(files):
    with open(f"{directory}/{file}", "r") as f:
        tmpdf = pd.DataFrame([json.loads(line) for line in f])
        # filter out the rows that do not have the target subreddit
        tmpdf = tmpdf[tmpdf["subreddit"].isin(target_subreddits)]
        # print(tmpdf.head())
        df = pd.concat([df, tmpdf[["subreddit", "author", "created_utc", "parent_id", "id", "body"]]])

        print(df.head())
        print('length:', len(df))
        df = df.sort_values(by="created_utc")

In [ ]:
print(df.head())

In [ ]:
df['id']

for id in df['id']:
    print(id)

In [ ]:
conversations = {}

for index, row in df.iterrows():
    latest_body = row["body"]
    latest_author = row["author"]
    latest_id = row["id"]

In [ ]:
#https://huggingface.co/docs/peft/quicktour 

from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

model = AutoPeftModelForCausalLM.from_pretrained("Sentdex/Walls1337bot-Llama2-7B-003.005.5000")
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

model = model.to("cuda")
model.eval()

In [ ]:
import colorama

for prompt in test_prompts:
    print(50*"=")
    formatted_prompt = f"### BEGIN CONVERSATION ###\n\n## Speaker_0: ##\n{prompt}\n\n## Walls1337bot: ##\n"
    inputs = tokenizer(formatted_prompt, return_tensors="pt")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=128)
    output_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
    just_response = output_text.split(formatted_prompt)[1].split("\n\n##")[0]
    
    #print(formatted_prompt+just_response)
    print(f"{colorama.Fore.CYAN}{formatted_prompt}{colorama.Style.RESET_ALL}{colorama.Fore.YELLOW}{just_response}{colorama.Style.RESET_ALL}")
    print(50*"=")
    print()
    print()